In [1]:

from utils.prompt import template
from utils.utils import process_pdfs_from_dataframe, process_pdfs_from_dataframe_faiss

import string
import pandas as pd 
from tqdm.autonotebook import tqdm

from utils.RagPipeline import Ragpipeline

from utils.utils import normalize_path

## Dense DB (Vector DB) 및 Sparse DB

test.csv에서 질문들에 매칭되는 소스(pdf) 정보가 주어진다. 

각 질문에 따른 소스에 접근을 하고 리트리버를 사용해도 되기 때문에 

각 질문에 따른 소스별 DB를 만들어주자

In [2]:
df = pd.read_csv('../open/train.csv')
base_directory = '../open' # Your Base Directory

df.head()

,SAMPLE_ID,Source,Source_path,Question,Answer
0,TRAIN_000,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?,"2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년..."
1,TRAIN_001,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요?,"2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7..."
2,TRAIN_002,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,기금이 예산과 다른 점은?,"기금은 예산과 구분되는 재정수단으로서 재정운영의 신축성을 기할 필요가 있을 때, 정..."
3,TRAIN_003,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,"일반회계, 특별회계, 기금 간의 차이점은 무엇인가요?","일반회계는 특정 사업 운영 및 특정 세입으로 특정 세출을 충당하는데 사용되고, 특별..."
4,TRAIN_004,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,"2024년 총수입은 얼마이며, 예산수입과 기금수입은 각각 몇 조원인가요?","2024년 총수입은 612.2조원이며, 예산수입은 395.5조원, 기금수입은 216..."


In [3]:
source_list = list(df['Source'].unique())

# 알파벳 리스트
alphabet = list(string.ascii_uppercase)

In [4]:
# 딕셔너리로 파일명과 알파벳 매핑
file_mapping = {alphabet[i]: source_list[i] for i in range(len(source_list))}

reversed_file_mapping = {value: key for key, value in file_mapping.items()}

file_mapping

{'A': '1-1 2024 주요 재정통계 1권',
 'B': '2024 나라살림 예산개요',
 'C': '재정통계해설',
 'D': '국토교통부_전세임대(융자)',
 'E': '고용노동부_청년일자리창출지원',
 'F': '고용노동부_내일배움카드(일반)',
 'G': '보건복지부_노인일자리 및 사회활동지원',
 'H': '중소벤처기업부_창업사업화지원',
 'I': '보건복지부_생계급여',
 'J': '국토교통부_소규모주택정비사업',
 'K': '국토교통부_민간임대(융자)',
 'L': '고용노동부_조기재취업수당',
 'M': '2024년도 성과계획서(총괄편)',
 'N': '「FIS 이슈 & 포커스」 23-3호 《조세지출 연계관리》',
 'O': '「FIS 이슈 & 포커스」 22-3호 《재정융자사업》',
 'P': '월간 나라재정 2023년 12월호'}

In [13]:
# PDF별 FAISS db 생성 
save_path = 'train_data'
os.makedirs(save_path, exist_ok=True)
process_pdfs_from_dataframe_faiss(df, base_directory, save_path, reversed_file_mapping)

가지고 있는 pdf파일들: ['./train_source/1-1 2024 주요 재정통계 1권.pdf'
 './train_source/2024 나라살림 예산개요.pdf' './train_source/재정통계해설.pdf'
 './train_source/국토교통부_전세임대(융자).pdf' './train_source/고용노동부_청년일자리창출지원.pdf'
 './train_source/고용노동부_내일배움카드(일반).pdf'
 './train_source/보건복지부_노인일자리 및 사회활동지원.pdf'
 './train_source/중소벤처기업부_창업사업화지원.pdf' './train_source/보건복지부_생계급여.pdf'
 './train_source/국토교통부_소규모주택정비사업.pdf' './train_source/국토교통부_민간임대(융자).pdf'
 './train_source/고용노동부_조기재취업수당.pdf' './train_source/2024년도 성과계획서(총괄편).pdf'
 './train_source/「FIS 이슈 & 포커스」 23-3호 《조세지출 연계관리》.pdf'
 './train_source/「FIS 이슈 & 포커스」 22-3호 《재정융자사업》.pdf'
 './train_source/월간 나라재정 2023년 12월호.pdf']


Processing PDFs:   0%|          | 0/16 [00:00<?, ?it/s]

Processing 1-1 2024 주요 재정통계 1권...


c:\Users\kim_h\anaconda3\envs\final\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
Processing PDFs:   6%|▋         | 1/16 [00:08<02:06,  8.42s/it]

Processing 2024 나라살림 예산개요...


...

Processing PDFs:  88%|████████▊ | 14/16 [01:07<00:09,  4.50s/it]

Processing 「FIS 이슈 & 포커스」 22-3호 《재정융자사업》...


Processing PDFs:  94%|█████████▍| 15/16 [01:12<00:04,  4.43s/it]

Processing 월간 나라재정 2023년 12월호...


Processing PDFs: 100%|██████████| 16/16 [01:16<00:00,  4.78s/it]


In [5]:
source = source_list[0]

target_df = df[df['Source'] == source][:10]

In [6]:
source

'1-1 2024 주요 재정통계 1권'

In [7]:
len(target_df)

10

In [8]:
pipeline = Ragpipeline(f'train_data/{reversed_file_mapping[source]}')

c:\Users\kim_h\anaconda3\envs\final\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [9]:
# 결과를 저장할 리스트 초기화
results = []

for _, row in tqdm(target_df.iterrows(), total=len(target_df), desc=f"Answering Questions"):
    # 소스 문자열 정규화
    source = normalize_path(row['Source'])
    
    # 질문 
    question = row['Question']
    print(f"Question: {question}")
    
    # 대답
    full_response = pipeline.answer_generation(question)
    first_line_response = full_response.split("\n")[0]
    print(f"Answer: {first_line_response}")
    
    # 정답 
    print(f"GT: {row['Answer']}\n")
    


Answering Questions:   0%|          | 0/10 [00:00<?, ?it/s]

Question: 2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?


Answering Questions:  10%|█         | 1/10 [00:27<04:10, 27.78s/it]

Answer: 2024년 중앙정부 재정체계는 일반회계, 특별회계, 기금으로 구분되며, 총수입은 612.2조원에 달하며, 예산수입 395.5조원, 기금수입 216.7조원으로 구성되어 있습니다.
GT: 2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년 기준으로 일반회계 1개, 특별회계 21개, 기금 68개로 구성되어 있습니다.

Question: 2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요?


Answering Questions:  20%|██        | 2/10 [00:44<02:51, 21.38s/it]

...

Answer: 2024년 국세수입은 367.3조원입니다. 일반회계는 356.1조원, 특별회계는 11.2조원이며, 일반회계 내국세수입은 321.6조원으로 전년도 예산 대비 36.4조원 감소했습니다.
GT: 2024년 국세수입은 367.3조원이며, 일반회계는 356.1조원, 특별회계는 11.2조원입니다.

Question: 2024년도 국세수입 중 일반회계 내국세수입은 몇 조원인가요?


Answering Questions:  90%|█████████ | 9/10 [03:08<00:19, 19.29s/it]

Answer: 2024년 국세수입 중 일반회계 내국세수입은 321.6조원입니다.
GT: 2024년도 일반회계 내국세수입은 321.6조원입니다.

Question: 2024년도 세외수입 규모와 구성은 어떤가요?


Answering Questions: 100%|██████████| 10/10 [03:36<00:00, 21.62s/it]

Answer: 2024년 세외수입은 총 28.2조원으로, 이는 전년보다 0.3% 증가했습니다. 세외수입의 구성은 경상이전수입 6.9조원, 재산수입 2.9조원, 기업특별회계영업수입 8.1조원 등으로 이루어졌습니다.
GT: 2024년 세외수입은 일반회계에서 11.2조원, 특별회계에서 17.0조원으로 나타났습니다.

